# Automated Generation of Training Data for Microsoft LUIS and Speech Service
This notebook serves to batch-generate training data for [Microsoft LUIS](https://luis.ai) and [Microsoft Speech Service](https://speech.microsoft.com) based on example utterances and possible entity-values.

## Example
### Input sentence: 
- "I would like to book a flight from {city} to {city} and my name is {name}."

### Sample values: 
- city: 'Stuttgart', 'Singapore', 'Frankfurt', 'Kuala Lumpur'
- name: 'Nadella', 'Gates'

### Returns:
- Training Data for Speech-To-Text Engine or textual input for Text-to-Speech generation
    - "I would like to book a flight from Frankfurt to Kuala Lumpur and my name is Nadella."
    - "I would like to book a flight from Singapore to Stuttgart and my name is Gates."
    - "I would like to book a flight from Singapore to Frankfurt and my name is Ballmer."
- Training data for Microsoft LUIS (see the concept of [LU-files](https://docs.microsoft.com/en-us/composer/concept-language-understanding))
    - I would like to book a flight from {city=Frankfurt} to {city=Kuala Lumpur} via {station=Bus Stop} and my name is {name=Nadella}.
    - I would like to book a flight from {city=Singapore} to {city=Stuttgart} via {station=Airport} and my name is {name=Gates}.
    - I would like to book a flight from {city=Singapore} to {city=Frankfurt} via {station=Airport} and my name is {name=Ballmer}.

In [23]:
# Import relevant packages
import json
import re
import logging
import pandas as pd
import random
import sys

# Import LUIS generator components
sys.path.append("../src/")
from luis_data_generator import LUISGenerator
from luis_data_generator import transform_lu

# Auto Reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# Define input values, or import them from a pandas data frame
utterances = ['i would like to book a flight from {city} to {city} via {station}, my name is {name}.', 
              'i am coming from {city} and want to travel via {station} to {city}.',
              'i want to book a seat on my flight to {city}.', 
              'how are you doing?']

values = {'city': ['Singapore', 'Frankfurt', 'Kuala Lumpur', 'Stuttgart'], 
          'station': ['Airport', 'Central Station', 'Bus Stop'], 
          'name': ['Nadella', 'Gates', 'Ballmer']}

intents =  ['BookFlight', 
            'BookFlight', 
            'BookSeat',
            'None']

In [31]:
# Create instance of the LUISGenerator-class along with your utterances, values and intents
# If you have no intents, just remove it. It is an optional argument for the class.
flight_generator = LUISGenerator(utterances, values, intents)

In [32]:
# Define amount of iterations below.
# Keep in mind that it does not necessarily mean, that there will be 1,000 examples of every utterance, as duplicates will be filtered out.
# The amount of utterances per example depends on the maximum number of combinations based on example-entity value combinations.
iterations = 1000

In [33]:
%%time
# Loop through the generator multiple times to get a variation of utterances
# If you have intents, speech_results and luis_results will be zipped lists each
# If you have no intents, speech_results and luis_results will be one-dimensional lists
speech_results = []
luis_results = []
for _ in range(1, iterations):
    flight_generator.get_values()
    speech, luis = flight_generator.fill_values()
    speech_results.extend(speech)
    luis_results.extend(luis)
print("Done!")

Done!
Wall time: 56.9 ms


## Speech to Text / Text to Speech
The section below give you a glance on the results and writes them to a text file.
If you write generated these utterances along with intents, you may also use it for LUIS scoring with GLUE, as you have intent-text combinations.
This can help you to evaluate the performance of the model given different entity values.


In [34]:
# Show the head of the speech-results
speech_results[:4]

[('BookFlight',
  'i would like to book a flight from Frankfurt to Stuttgart via Airport, my name is Nadella.'),
 ('BookFlight',
  'i am coming from Kuala Lumpur and want to travel via Airport to Singapore.'),
 ('BookSeat', 'i want to book a seat on my flight to Stuttgart.'),
 ('None', 'how are you doing?')]

## LUIS
The section below shows you how the results look like and writes them to a [LU-files](https://docs.microsoft.com/en-us/composer/concept-language-understanding). This file can be used as input file for [LUIS](https://luis.ai) training and to accelerate your model development.

In [35]:
# Show the head of the luis results
luis_results[:5]

[('BookFlight',
  'i would like to book a flight from {city=Frankfurt} to {city=Stuttgart} via {station=Airport}, my name is {name=Nadella}.'),
 ('BookFlight',
  'i am coming from {city=Kuala Lumpur} and want to travel via {station=Airport} to {city=Singapore}.'),
 ('BookSeat', 'i want to book a seat on my flight to {city=Stuttgart}.'),
 ('None', 'how are you doing?'),
 ('BookFlight',
  'i would like to book a flight from {city=Stuttgart} to {city=Frankfurt} via {station=Central Station}, my name is {name=Gates}.')]

In [36]:
# File name of your target LU-file.
file_name = 'example_lu_file' 
# Boolean to write to file, if false it will only show in the output.
write = True
# Transform to LU-file. Keep in mind, that you will need a list of tuples with intents, otherwise the function will throw an error.
transform_lu(luis_results, file_name, write=True)


# BookFlight
- i would like to book a flight from {city=Frankfurt} to {city=Stuttgart} via {station=Airport}, my name is {name=Nadella}.
- i would like to book a flight from {city=Stuttgart} to {city=Singapore} via {station=Central Station}, my name is {name=Nadella}.
- i am coming from {city=Stuttgart} and want to travel via {station=Airport} to {city=Kuala Lumpur}.
- i would like to book a flight from {city=Stuttgart} to {city=Kuala Lumpur} via {station=Bus Stop}, my name is {name=Gates}.
- i am coming from {city=Kuala Lumpur} and want to travel via {station=Central Station} to {city=Stuttgart}.
- i would like to book a flight from {city=Frankfurt} to {city=Stuttgart} via {station=Central Station}, my name is {name=Nadella}.
- i am coming from {city=Frankfurt} and want to travel via {station=Airport} to {city=Stuttgart}.
- i would like to book a flight from {city=Kuala Lumpur} to {city=Singapore} via {station=Bus Stop}, my name is {name=Ballmer}.
- i am coming from {city=Stuttgart} 